In [145]:
pip install datasets evaluate transformers rouge-score nltk

In [146]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OMEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [147]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\OMEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [148]:
from transformers.utils import send_example_telemetry

send_example_telemetry("summarization_notebook", framework="pytorch")

In [149]:
model_checkpoint = "t5-small"

In [150]:
from datasets import load_dataset
from evaluate import load

raw_datasets = load_dataset("xsum")
metric = load("rouge")

In [151]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [11]:
raw_datasets["train"][0]

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

### Random Samples from the dataset

In [152]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [153]:
show_random_elements(raw_datasets["train"])

,document,summary,id
0,"Private Harry Vasey, who was part of the 1st Airborne Battalion, The Border Regiment, was killed during Operation Market Garden in Oosterbeek in 1944.\nNow his identity has been confirmed, the Ministry of Defence (MoD) want to trace his family so his grave can be rededicated in the Netherlands.\nThe MoD said plans were also in place to change his headstone.\nBorn in Durham in May 1916 to Harry Vasey and Annie Young, he enlisted in April 1940 when he lived in Bowburn, County Durham.\nAn MoD spokesman said: ""Unfortunately that is about all we know about Private Vasey and his family and that's where the trail goes cold.\n""We are hoping that there are some of his family still living in that area.""\nSince WW2, a section of the Royal Netherlands Army has been working to identify the graves of unknown soldiers killed in battle.\nThe exhumation reports were scrutinised for clues to the identities of these men and the research was presented to the MoD.\nMr Vasey is one of six Border Regiment soldiers, including Lance Corporal Raymond Halliday, to be indentified.\nThe aim of Operation Market Garden was to take strategic bridges near Arnhem, but the Allies underestimated the number of German troops lying in wait and it failed.\nMore than 1,400 Allied troops died and more than 6,000 were captured by German forces.\nIt is hoped Mr Vasey's surviving relatives can attend the service at Oosterbeek Cemetery on 14 September in honour of his sacrifice and bravery, the MoD said.",The family of a soldier killed during World War Two is being sought after his final resting place was confirmed.,36802710
1,"Hooker Richard Hibbard and prop John Afoa return as David Humphreys makes 10 changes to the team which beat the French side 35-14 on 8 December.\nLewis Ludlow, Jacob Rowan and Ben Morgan also start in the pack, while Callum Braley starts at scrum-half.\nJames Hook takes David Halaifonua's spot at full-back, with Jonny May and Charlie Sharples on the wings.\nStade Rochelais: Murimurivalu; Lacroix, Jordaan, Aguillon, Rattez; James, Bales; Corbel, Forbes, Atonio, Qovu, Tanguy, Sazy, Eaton (capt.), Vito.\nReplacements: Maurouard, Priso, Boughanmi, Cedaro, Francoz, Retiere, Holmes, Botia.\nGloucester: Hook; Sharples, Scott, Atkinson, May; Burns, Braley (capt.); Hohneck, Hibbard, Afoa, Savage, Thrush, Ludlow, Rowan, Morgan.\nReplacements: Matu'u, Thomas, Doran-Jonesm Latta, Galarza, Heinz, Purdy, Kvesic.\nFor the latest rugby union news follow @bbcrugbyunion on Twitter.",Gloucester lock Jeremy Thrush will make his first appearance of the season against Stade Rochelais.,38307773
2,"The university announced back in March it needed to save £10.5m and planned to cut 150 posts.\nMembers of the University and College Union (UCU) Scotland were balloted as a result.\nAfter the ballot closed on Monday, the union said 73% of those who voted backed strike action. And 80% also voted for action short of a strike.\nAndrew MacKillop, Aberdeen UCU representative, said: ""Members have made it quite clear that they reject the job losses proposed by the university.\n""Strike action is always a last resort but we can't sit back and see jobs lost with the accompanying damage to the student experience and the reputation of the university.""\nIn a statement, the university said it was disappointed that the union had vote for strike action ""in the midst of ongoing dialogue"".\nIt added: ""According to the results of the ballot, 263 UCU members voted in favour of strike action, representing 12.5% of our total academic and academic-related workforce.\n""The UCU had asked for assurance that the university would rule out compulsory redundancies as it seeks to make savings of £10.5m.\n""We were unable to give that assurance, although we are working tirelessly to achieve the savings we need through voluntary measures as far as possible, and are pursuing a range of additional options to increase our efficiency as a world-leading university.""",Staff at the 

In [14]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [154]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

### Tokenization and Preprocessing

In [155]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [156]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [157]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [158]:
print(tokenizer(text_target=["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


In [159]:
if model_checkpoint in ["t5-small", "t5-base", "t5-large", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [160]:
max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [161]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[21603, 10, 37, 423, 583, 13, 1783, 16, 20126, 16496, 6, 80, 13, 8, 844, 6025, 4161, 6, 19, 341, 271, 14841, 5, 7057, 161, 19, 4912, 16, 1626, 5981, 11, 186, 7540, 16, 1276, 15, 2296, 7, 5718, 2367, 14621, 4161, 57, 4125, 387, 5, 15059, 7, 30, 8, 4653, 4939, 711, 747, 522, 17879, 788, 12, 1783, 44, 8, 15763, 6029, 1813, 9, 7472, 5, 1404, 1623, 11, 5699, 277, 130, 4161, 57, 18368, 16, 20126, 16496, 227, 8, 2473, 5895, 15, 147, 89, 22411, 139, 8, 1511, 5, 1485, 3271, 3, 21926, 9, 472, 19623, 5251, 8, 616, 12, 15614, 8, 1783, 5, 37, 13818, 10564, 15, 26, 3, 9, 3, 19513, 1481, 6, 18368, 186, 1328, 2605, 30, 7488, 1887, 3, 18, 8, 711, 2309, 9517, 89, 355, 5, 3966, 1954, 9233, 15, 6, 113, 293, 7, 8, 16548, 13363, 106, 14022, 84, 47, 14621, 4161, 6, 243, 255, 228, 59, 7828, 8, 1249, 18, 545, 11298, 1773, 728, 8, 8347, 1560, 5, 611, 6, 255, 243, 72, 1709, 1528, 161, 228, 43, 118, 4006, 91, 12, 766, 8, 3, 19513, 1481, 410, 59, 5124, 5, 96, 196, 17, 19, 1256, 68, 27, 103, 317, 132

In [162]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

In [163]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


### Model Setup and Training Arguments

In [121]:
device = torch.device("cpu")

In [164]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint) # Google Colab (GPU)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to(device)

In [165]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [100]:
pip install bitsandbytes

   ---------------------------------------- 0.0/68.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/68.5 MB ? eta -:--:--
   ---------------------------------------- 0.8/68.5 MB 2.8 MB/s eta 0:00:25
    --------------------------------------- 1.3/68.5 MB 2.6 MB/s eta 0:00:27
   - -------------------------------------- 1.8/68.5 MB 2.6 MB/s eta 0:00:26
   - -------------------------------------- 2.4/68.5 MB 2.6 MB/s eta 0:00:26
   - -------------------------------------- 2.9/68.5 MB 2.6 MB/s eta 0:00:26
   -- ------------------------------------- 3.7/68.5 MB 2.7 MB/s eta 0:00:24
   -- ------------------------------------- 4.5/68.5 MB 2.9 MB/s eta 0:00:23
   --- ------------------------------------ 5.2/68.5 MB 2.9 MB/s eta 0:00:22
   --- ------------------------------------ 6.0/68.5 MB 3.0 MB/s eta 0:00:21
   --- ------------------------------------ 6.6/68.5 MB 3.0 MB/s eta 0:00:21
   ---- ----------------------------------- 7.6/68.5 MB 3.2 MB/s eta 0:00:20
   ---- -----

In [ ]:
# Saving model in Google drive
from google.colab import drive
drive.mount('/content/drive')

model_dir = "/content/drive/My Drive/Models/t5-summarization"

In [ ]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    output_dir=model_dir,  # Save to Google Drive
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    report_to=["none"],
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    # save_steps=500,  # Save checkpoint every 500 steps
    # logging_dir='./logs',  # Directory to store logs
    # logging_steps=100  # Log metrics every 100 steps
    # use_cpu=True,  # Force CPU usage
)

### Evaluation Metric (ROUGE)

In [167]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [168]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

### Fine-Tune the Model

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.train(resume_from_checkpoint=True)

In [ ]:
trainer.save_model(model_dir)  # This saves both the model and the tokenizer
tokenizer.save_pretrained(model_dir)

In [ ]:
# Loading the model from Google Drive to use it in Google Colab
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_dir = "/content/drive/My Drive/Models/t5-summarization"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

In [ ]:
!zip -r model.zip {model_dir}
from google.colab import files
files.download("model.zip")

In [174]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Path to the final saved model
model_dir = "C:\\Users\\OMEN\\BrevityBot\\t5-summarization-final"

# Load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [175]:
text = """
Many financial institutions started building conversational AI, prior to the Covid19
pandemic, as part of a digital transformation initiative. These initial solutions
were high profile, highly personalized virtual assistants — like the Erica chatbot
from Bank of America. As the pandemic hit, the need changed as contact centers were
under increased pressures. As Cathal McGloin of ServisBOT explains in “how it started,
and how it is going,” financial institutions were looking for ways to automate
solutions to help get back to “normal” levels of customer service. This resulted
in a change from the “future of conversational AI” to a real tactical assistant
that can help in customer service. Haritha Dev of Wells Fargo, saw a similar trend.
Banks were originally looking to conversational AI as part of digital transformation
to keep up with the times. However, with the pandemic, it has been more about
customer retention and customer satisfaction. In addition, new use cases came about
as a result of Covid-19 that accelerated adoption of conversational AI. As Vinita
Kumar of Deloitte points out, banks were dealing with an influx of calls about new
concerns, like questions around the Paycheck Protection Program (PPP) loans. This
resulted in an increase in volume, without enough agents to assist customers, and
tipped the scale to incorporate conversational AI. When choosing initial use cases
to support, financial institutions often start with high volume, low complexity
tasks. For example, password resets, checking account balances, or checking the
status of a transaction, as Vinita points out. From there, the use cases can evolve
as the banks get more mature in developing conversational AI, and as the customers
become more engaged with the solutions. Cathal indicates another good way for banks
to start is looking at use cases that are a pain point, and also do not require a
lot of IT support. Some financial institutions may have a multi-year technology
roadmap, which can make it harder to get a new service started. A simple chatbot
for document collection in an onboarding process can result in high engagement,
and a high return on investment. For example, Cathal has a banking customer that
implemented a chatbot to capture a driver’s license to be used in the verification
process of adding an additional user to an account — it has over 85% engagement
with high satisfaction. An interesting use case Haritha discovered involved
educating customers on financial matters. People feel more comfortable asking a
chatbot what might be considered a “dumb” question, as the chatbot is less judgmental.
Users can be more ambiguous with their questions as well, not knowing the right
words to use, as chatbot can help narrow things down.
"""

inputs = ["summarize: " + text]
inputs = tokenizer(inputs, max_length=512, truncation=True, return_tensors="pt")

outputs = model.generate(**inputs, max_length=64, num_beams=4)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(summary)

Financial institutions were looking for ways to automate the adoption of conversational AI as part of a digital transformation initiative.
